In [1]:
import os
from openai import OpenAI
from langchain_openai import ChatOpenAI
import sys
import utils
import base64
import pandas as pd

In [2]:
client = OpenAI()

In [3]:
# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

In [41]:
road_grading_standards = """Road Condition Rating System (Pavement Surface Condition Index - PSCI):
    Rating 10 (Excellent Road Condition - Routine Maintenance): Newly constructed or recently overlaid roads with no visible defects.
    Rating 9 (Very Good Road Condition - Routine Maintenance): Very few visible defects; less than 10% surface defects (ravelling and bleeding).
    Rating 8 (Good Road Condition - Restoration of Skid Resistance): 10% to 30% surface with surface defects; no structural distresses present.
    Rating 7 (Good Road Condition - Restoration of Skid Resistance): More than 30% of surface with surface defects (ravelling or bleeding); no structural distresses.
    Rating 6 (Fair Road Condition - Surface Restoration): Less than 20% other cracking, patching in good condition, some surface distortion present.
    Rating 5 (Fair Road Condition - Surface Restoration): More than 20% surface with other cracking, patching in fair condition, significant surface distortions.
    Rating 4 (Poor Road Condition - Structural Overlay): 5% to 25% of surface with structural distresses (rutting, alligator cracking, poor patching), frequent potholes.
    Rating 3 (Poor Road Condition - Structural Overlay): 25% to 50% of surface with significant structural distresses, more frequent potholes.
    Rating 2 (Very Poor  Road Condition - Road Reconstruction): More than 50% of surface with severe structural distresses, severe rutting, many potholes.
    Rating 1 (Failed Poor  Road Condition - Road Reconstruction): Complete road failure, many large and deep potholes, extensive loss of pavement surface.
    
    Defect Categories and Indicators:
    Surface Defects: Include ravelling (loss of binder and aggregate from the surface) and bleeding (excess bituminous material on the surface).
    Pavement Deformation: Encompasses rutting (permanent deformation in wheel paths) and surface distortion (depressions, bumps).
    Cracks: Categorized into alligator cracking (series of interconnected cracks), edge breakup/cracking, and other cracking (longitudinal, transverse, reflection, slippage).
    Surface Openings: Include patching (repaired areas indicating previous pavement defects) and potholes (bowl-shaped depressions exposing underlying layers).
"""

In [42]:
def get_road_condition_grade1(folder_path, model):
    # Iterate through each image in the folder
    grade = []
    img_names = []
    for image_name in os.listdir(folder_path):
        image_path = os.path.join(folder_path, image_name)

        if os.path.isfile(image_path) and image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
            base64_image = encode_image(image_path)
            # Construct the message with the specified instructions
            messages=[
                      {
                        "role": "system",
                        "content":"You are a pavement engineer.Please provide a grade for the road condition based on the\
                             provided standards and image. Response me with only one number(grade)"},
                      {
                        "role": "user",
                        "content": [
                            { "type": "image_url","image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}},
                            {"type": "text", "text":  "Please pay attention to any anomalies on the asphalt pavement"},
                            {"type": "text", "text":  road_grading_standards},
                            {"type": "text", "text": "Using the provided road condition grading criteria determine the \
                            grade of the road surface in the image"},

                        ]
                    },
                ]
            
            response = client.chat.completions.create(
                model= model,  # Note: This is a conceptual model name
                messages=messages,
                max_tokens=300,
                temperature=0
            )
            
            try:
                grade_value = int(response.choices[0].message.content)
                grade.append(grade_value)
                img_names.append(image_name)
            except ValueError:
                grade_value = 11
                grade.append(grade_value)
                img_names.append(image_name)
                print(f"Invalid response for {image_name}: {response.choices[0].message.content}")
    return (img_names, grade)

In [43]:
folder_path = r"C:\Desktop\P-Roadphone\Paper_LLM\FINAL\imgs_all"
output = get_road_condition_grade1(folder_path,"gpt-4o")

df = pd.DataFrame({'Name': output[0], 'Model': output[1]})
df['Name'] = df['Name'].str.extract('(\d+)')
csv_path = r"C:\Desktop\P-Roadphone\Paper_LLM\FINAL\RC_Model3_o_09211606.csv"#RC_Model2_o6_08252333.csv
df.to_csv(csv_path, index=False)